# Starfish BaristaSeq Processing Example

In [1]:
%gui qt

import os
from copy import deepcopy
from itertools import product

import numpy as np
import pandas as pd
import skimage.filters
import skimage.morphology
from skimage.transform import SimilarityTransform, warp
from tqdm import tqdm

import starfish
import starfish.data
from starfish.spots import SpotFinder
from starfish.types import Axes

BaristaSeq is an assay that sequences padlock-probe initiated rolling circle amplified spots using a one-hot codebook. The publication for this assay can be found [here](https://www.ncbi.nlm.nih.gov/pubmed/29190363).

The first step in BaristaSeq is to do some rough registration. For this data, the rough registration has been done for us by the authors. 

In [37]:
starfish.display(img)

In [24]:
exp = starfish.Experiment.from_json(os.path.expanduser("~/scratch/baristaseq/experiment.json"))
nissl = exp['fov_000'].get_image('dots')

100%|██████████| 17/17 [00:00<00:00, 61.49it/s]


In [25]:
# exp = starfish.data.BaristaSeq()
img = exp['fov_000'].get_image('primary')

100%|██████████| 476/476 [00:07<00:00, 61.99it/s]


In [26]:
starfish.display(img)

Analysis of BaristaSeq is done in 2-d

In [26]:
starfish.display(nissl)

In [29]:
z_projected_image = img.max_proj(Axes.ZPLANE)
z_projected_nissl = nissl.max_proj(Axes.ZPLANE)

100%|██████████| 1/1 [00:00<00:00, 177.65it/s]


There is a slight miss-alignment of the C channel in the microscope used to process the data. We transform the data to account for this. 

In [30]:
transform = SimilarityTransform(translation=(1.9, -0.4))
channels = (0,)
rounds = np.arange(img.num_rounds)
slice_indices = product(channels, rounds)

for ch, round_, in slice_indices:
    selector = {Axes.ROUND: round_, Axes.CH: ch, Axes.ZPLANE: 0}
    tile = z_projected_image.get_slice(selector)[0]
    transformed = warp(tile, transform)
    z_projected_image.set_slice(
        selector=selector,
        data=transformed.astype(np.float32),
    )

In [31]:
from skimage.feature import register_translation  # noqa
from skimage.transform import warp  # noqa
from skimage.transform import SimilarityTransform  # noqa
from functools import partial

def _register_imagestack(target_image, reference_image, upsample_factor=5):
    target_image = np.squeeze(target_image)
    reference_image = np.squeeze(reference_image)
    shift, error, phasediff = register_translation(target_image, reference_image, upsample_factor=1)
    return SimilarityTransform(translation=shift)

projection = z_projected_image.max_proj(Axes.CH, Axes.ZPLANE)
reference_image = projection.sel({Axes.ROUND: 1}).xarray

register_imagestack = partial(
    _register_imagestack, reference_image=reference_image, upsample_factor=5
)
transforms = projection.transform(register_imagestack, group_by={Axes.ROUND}, n_processes=1)
round_to_tf = {axes[Axes.ROUND]: tf for tf, axes in transforms}

100%|██████████| 7/7 [00:00<00:00, 188.77it/s]


In [32]:
starfish.display(projection)

In [33]:
[t.translation for t in round_to_tf.values()]

[array([ 1., -2.]),
 array([0., 0.]),
 array([ 0., -1.]),
 array([ 1., -2.]),
 array([ 2., -2.]),
 array([ 2., -3.]),
 array([ 2., -2.])]

In [35]:
# starfish doesn't have a great ability to apply different functions to different parts of an
# imagestack based on their Axes. It would be great if one could iterate over the ImageStack
# and apply a set of transformations to each tile based on the tile's coordinates.

def _warp_tile(tile, transform):
    round_ = int(tile.coords[Axes.ROUND.value])
    return warp(np.squeeze(tile), transforms[round_])

def warp_imagestack(imagestack, transform_set, chunk_by):

    new = starfish.ImageStack.from_numpy_array(np.zeros_like(imagestack.xarray.values))

    selectors = product(*(list(int(i) for i in imagestack.xarray.coords[c]) for c in chunk_by))
    for s in selectors:
        selector = dict(zip(chunk_by, s))
        data = np.squeeze(imagestack.xarray.sel(selector))
        res = warp(data, transform_set[selector[Axes.ROUND.value]])
        new.set_slice(selector, res.astype(np.float32))

    return new


transformed = warp_imagestack(z_projected_image, round_to_tf, chunk_by=("r", "c", "z"))

100%|██████████| 28/28 [00:00<00:00, 195.85it/s]


In [36]:
starfish.display(transformed)

High-pass filter to remove camera noise. 

In [5]:
ghp = starfish.image.Filter.GaussianHighPass(sigma=1)
high_passed = ghp.run(z_projected_image, in_place=False)

12it [00:00, 266.95it/s]


Correct for bleed-through from Illumina SBS reagents

In [6]:
# TODO this can be replaced with Kevin's linear unmixing!

In [38]:
bleed_correction_factors = pd.DataFrame(
    data=[
        [0, 1, 0.05],
        [0, 2, 0],
        [0, 3, 0],
        [1, 0, 0.35],
        [1, 2, 0],
        [1, 3, 0],
        [2, 0, 0],
        [2, 1, 0.02],
        [2, 3, 0.84],
        [3, 0, 0],
        [3, 1, 0],
        [3, 2, 0.05]
    ],
    columns=(('bleed_from', 'bleed_into', 'factor_bleed_from_into')),
)


def do_bleed_correction(stack):
    bleed_corrected = deepcopy(stack)

    for index, (ch1, ch2, constant) in tqdm(bleed_correction_factors.iterrows()):
        bleed = stack.get_slice({Axes.CH: int(ch1)})[0] * constant
        img_to_correct = stack.get_slice({Axes.CH: int(ch2)})[0]
        corrected = np.maximum(img_to_correct - bleed, 0)
        bleed_corrected.set_slice(
            {Axes.CH: int(ch2)},
            corrected,
            axes=[Axes.ROUND, Axes.ZPLANE]
        )
    return bleed_corrected


bleed_corrected = do_bleed_correction(transformed)


12it [00:00, 16.75it/s]


Remove image background

In [8]:
starfish.display(bleed_corrected)

In [39]:
clip = starfish.image.Filter.Clip(p_min=90, p_max=99.9)
clipped = clip.run(bleed_corrected)

In [40]:
starfish.display(clipped)

In [42]:
wth = starfish.image.Filter.WhiteTophat(masking_radius=10)

background_corrected = wth.run(bleed_corrected, in_place=False)

In [47]:
mh = starfish.image.Filter.MatchHistograms(group_by={Axes.CH, Axes.ROUND})
matched = mh.run(background_corrected)

In [48]:
starfish.display(matched)

In [111]:
threshold = np.percentile(matched.xarray.values, 96)

In [112]:
lsbd = starfish.spots._detector.local_search_blob_detector.LocalSearchBlobDetector(
    min_sigma=(1, 1, 1),
    max_sigma=(8, 8, 8),
    num_sigma=10,
    threshold=threshold,
    search_radius=7
)
intensities = lsbd.run(matched)

In [113]:
starfish.display(matched, intensities)

In [94]:
intensities

<xarray.IntensityTable (features: 1076, c: 4, r: 7)>
array([[[0.001821, 0.000753, ..., 0.002201,      nan],
        [     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan, 0.001116]],

       [[     nan, 0.000878, ...,      nan,      nan],
        [     nan,      nan, ..., 0.001201,      nan],
        [0.002012,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan, 0.002224]],

       ...,

       [[0.001072,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan, 0.001083],
        [     nan,      nan, ...,      nan,      nan],
        [     nan, 0.001079, ...,      nan,      nan]],

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        [     nan, 0.000947, ..., 0.000945,      nan]]])
Coordinates:
    radius   (features) float64 6

In [115]:
?exp.codebook.decode_per_round_max

In [98]:
decoded = exp.codebook.decode_per_round_max(intensities.fillna(0))

In [114]:
np.sum(~(decoded['target'] == 'nan'))

<xarray.DataArray 'target' ()>
array(92)

In [61]:
test_arr = matched.xarray[0, 0, 0].values
from skimage.feature import blob_log

In [65]:
blob_log(test_arr, threshold=90)

array([], shape=(0, 3), dtype=float64)